## semantic segmentation
- `semantic segmentation`이란 한 장의 화상에 포함된 여러 물체의 영역과 이름을 픽셀 수준에서 지정하느 작업이다.
- obejcet detection이 물체를 커다란 직사각형의 BBox로 묶었지만 시맨틱 분할에서는 픽셀 수준으로 **어디에서 어디까지 어떠한 클래스 물체인지** 라벨을 붙입니다. 
- 제조업의 흠집 탐지, 의료 영상 진단의 병변 감지, 자율 운전의 주변 환경 파악 등에서 시맨틱 분할 기술을 사용합니다. 
- 입력 : 이미지
- 출력 : 각 픽셀이 속한 클래스의 라벨 정보
- semantic segmentation 순서
    1. 이미지 전처리(리사이즈, 정규화)
    2. 이미지를 pspnet에입력 
    3. pspnet 출력이 최댓값인 클래스 추출 
        - (클래스수 x 475 x 475)의 배열 -> (475 x 475)의 배열
    4. 3단계의 출력(475 x 475) 배열을 원본 이미지 크기로 복원 


# code

In [ ]:
!git clone https://github.com/YutaroOgawa/pytorch_advanced.git

Cloning into 'pytorch_advanced'...
remote: Enumerating objects: 552, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 552 (delta 36), reused 61 (delta 31), pack-reused 479
Receiving objects: 100% (552/552), 17.80 MiB | 31.98 MiB/s, done.
Resolving deltas: 100% (295/295), done.


In [ ]:
cd pytorch_advanced/3_semantic_segmentation/

/content/pytorch_advanced/3_semantic_segmentation


In [ ]:
import os
import urllib.request
import zipfile
import tarfile

# フォルダ「data」が存在しない場合は作成する
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

    # VOC2012のデータセットをここからダウンロードします
# 時間がかかります（約15分）
url = "http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar"
target_path = os.path.join(data_dir, "VOCtrainval_11-May-2012.tar") 

if not os.path.exists(target_path):
    urllib.request.urlretrieve(url, target_path)
    
    tar = tarfile.TarFile(target_path)  # tarファイルを読み込み
    tar.extractall(data_dir)  # tarを解凍
    tar.close()  # tarファイルをクローズ
    

# Dataloader

In [ ]:
# 라이브러리 임포트
import os.path as osp
from PIL import Image

import torch.utils.data as data

In [ ]:
def make_datapath_list(rootpath):
    """
    학습 및 검증용 화상 데이터와 어노테이션 데이터의 파일 경로 리스트 작성

    Parameters
    ----------
    rootpath : str
        데이터의 폴더 경로

    Returns
    -------
    ret : train_img_list, train_anno_list, val_img_list, val_anno_list
        데이터 경로를 저장한 리스트
    """

    # 이미지 파일과 어노테이션 파일의 경로 템플릿 작성
    imgpath_template = osp.join(rootpath, 'JPEGImages', '%s.jpg')
    annopath_template = osp.join(rootpath, 'SegmentationClass', '%s.png')

    # 후련 및 검증 파일 각각의 ID(파일 이름) 취득
    train_id_names = osp.join(rootpath + 'ImageSets/Segmentation/train.txt')
    val_id_names = osp.join(rootpath + 'ImageSets/Segmentation/val.txt')

    # 훈련 데이터의 이미지 파일과 어노테이션 파일의 경로 리스트 작성
    train_img_list = list()
    train_anno_list = list()

    for line in open(train_id_names):
        file_id = line.strip()  # 공백과 줄 바꿈 제거
        img_path = (imgpath_template % file_id)  # 이미지 경로
        anno_path = (annopath_template % file_id)  # 어노테이션 경로
        train_img_list.append(img_path)
        train_anno_list.append(anno_path)

    # 검증 데이터의 이미지 파일과 어노테이션 파일의 경로 리스트 작성
    val_img_list = list()
    val_anno_list = list()

    for line in open(val_id_names):
        file_id = line.strip()  # 공백과 줄 바꿈
        img_path = (imgpath_template % file_id)  # 이미지 경로
        anno_path = (annopath_template % file_id)  # 어노테이션 경로
        val_img_list.append(img_path)
        val_anno_list.append(anno_path)

    return train_img_list, train_anno_list, val_img_list, val_anno_list

In [ ]:
# 실행 확인
rootpath = "./data/VOCdevkit/VOC2012/"

train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath=rootpath)

print(train_img_list[0])
print(train_anno_list[0])

./data/VOCdevkit/VOC2012/JPEGImages/2007_000032.jpg
./data/VOCdevkit/VOC2012/SegmentationClass/2007_000032.png


In [ ]:
# 데이터 처리 클래스와 데이터 확장 클래스
from utils.data_augumentation import Compose, Scale, RandomRotation, RandomMirror, Resize, Normalize_Tensor


class DataTransform():
    """
    이미지와 어노테이션의 전처리 클래스. 학습과 검증 시 다르게 작동.
    이미지 크기를 input_size x input_size로 한다.
    검증 시 데이터 확장을 수행 한다.


    Attributes
    ----------
    input_size : int
        리사이즈 대상의 이미지 크기
    color_mean : (R, G, B)
        각 색상 채널의 평균값
    color_std : (R, G, B)
        각 색상 채널의 표준편차
    """

    def __init__(self, input_size, color_mean, color_std):
        self.data_transform = {
            'train': Compose([
                Scale(scale=[0.5, 1.5]),  # 이미지 확대
                RandomRotation(angle=[-10, 10]),  # 회전
                RandomMirror(),  # 랜덤 미러
                Resize(input_size),  # 리사이즈(input_size)
                Normalize_Tensor(color_mean, color_std)  # 색상 정보의 표준화와 텐서로 변환
            ]),
            'val': Compose([
                Resize(input_size),  # 리사이즈(input_size)
                Normalize_Tensor(color_mean, color_std)  # 색상 정보의 표준화와 텐서로 변환
            ])
        }

    def __call__(self, phase, img, anno_class_img):
        """
        Parameters
        ----------
        phase : 'train' or 'val'
            전처리 모드 지정
        """
        return self.data_transform[phase](img, anno_class_img)

In [ ]:
class VOCDataset(data.Dataset):
    """
    Voc2012의 Dataset을 만드는 클래스. 파이토치의 Dataset 클래스를 상속 받는다.


    Attributes
    ----------
    img_list : list
        어노테이션 경로를 저장한 리스트
    anno_list : lsit
        어노테이션 경로를 저장한 리스트
    phase : 'train' or 'test'
        학습 또는 훈련 지정
    transform : object
        전처리 클래스 인스턴스
    """

    def __init__(self, img_list, anno_list, phase, transform):
        self.img_list = img_list
        self.anno_list = anno_list
        self.phase = phase
        self.transform = transform

    def __len__(self):
        '''이미지 갯수 반환'''
        return len(self.img_list)

    def __getitem__(self, index):
        '''
        전처리한 화상의 텐서 형식 데이터와 어노테이션 휙득
        '''
        img, anno_class_img = self.pull_item(index)
        return img, anno_class_img

    def pull_item(self, index):
        '''이미지 텐서 형식 데이터와 어노테이션 휙득'''

        # 1. 이미지 읽기
        image_file_path = self.img_list[index]
        img = Image.open(image_file_path)   # [높이][폭][색RGB]

        # 2. 어노테이션 이미지 읽기 
        anno_file_path = self.anno_list[index]
        anno_class_img = Image.open(anno_file_path)   # [높이][폭]

        # 3. 전처리 적용
        img, anno_class_img = self.transform(self.phase, img, anno_class_img)

        return img, anno_class_img

In [ ]:
# 동작 확인

# (RGB)색의 평균치와 표준 편차
color_mean = (0.485, 0.456, 0.406)
color_std = (0.229, 0.224, 0.225)

# 데이터셋 작성
train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(
    input_size=475, color_mean=color_mean, color_std=color_std))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size=475, color_mean=color_mean, color_std=color_std))

# 데이터 추출 예
print(val_dataset.__getitem__(0)[0].shape)
print(val_dataset.__getitem__(0)[1].shape)
print(val_dataset.__getitem__(0))

torch.Size([3, 475, 475])
torch.Size([475, 475])
(tensor([[[ 1.6667,  1.5125,  1.5639,  ...,  1.7523,  1.6667,  1.7009],
         [ 1.5810,  1.4269,  1.4783,  ...,  1.7009,  1.6153,  1.6495],
         [ 1.5639,  1.4098,  1.4440,  ...,  1.6838,  1.5982,  1.6324],
         ...,
         [-0.4739, -0.4911, -0.5424,  ...,  1.2557,  1.1872,  1.2214],
         [-0.5596, -0.4911, -0.4911,  ...,  1.2385,  1.1872,  1.2214],
         [-0.6281, -0.3883, -0.3369,  ...,  1.2385,  1.1872,  1.2214]],

        [[ 1.8333,  1.6758,  1.7283,  ...,  1.9209,  1.8333,  1.8683],
         [ 1.7458,  1.5882,  1.6408,  ...,  1.8683,  1.7808,  1.8158],
         [ 1.7283,  1.5707,  1.6057,  ...,  1.8508,  1.7633,  1.7983],
         ...,
         [-0.5826, -0.6001, -0.6527,  ...,  1.4132,  1.3431,  1.3431],
         [-0.6702, -0.6001, -0.6001,  ...,  1.3957,  1.3431,  1.3431],
         [-0.7402, -0.4951, -0.4426,  ...,  1.3957,  1.3431,  1.3431]],

        [[ 2.0474,  1.8905,  1.9428,  ...,  2.1346,  2.0474,  2.08

# 데이터 로더 작성

In [ ]:
# 데이터로더작성

batch_size = 8

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# dict로 오브젝트 정리
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

# 작동 확인
batch_iterator = iter(dataloaders_dict["val"])  # iterator로 변환
imges, anno_class_imges = next(batch_iterator)  # 첫 번째 요소르 꺼냄
print(imges.size())  # torch.Size([8, 3, 475, 475])
print(anno_class_imges.size())  # torch.Size([8, 3, 475, 475])

torch.Size([8, 3, 475, 475])
torch.Size([8, 475, 475])


# PSPNet 구성
---
PSPNet은 4개의 모듈인 `Feature, Pyramid Pooling, Decoder, AuxLoss`로 구성 되어 있다.
1. Feature 모듈 (Encoder 모듈)
    - 입력 이미지의 특징을 파악
    - 출력 : 2048 x 60 x 60 (채널 x 높이 x 폭)
    - 이미지의 특징을 파악한 2048ch과 이미지 크기가 60x60으로 변환됨.
2. Pyramid Pooling 모듈 
    - 어떠한 픽셀의 물체 라벨을 구하려면 다양한 크기로 해당 픽셀 주변 정보가 필요
    - Pyramid Pooling 모듈에서는 네 가지 크기의 특징량 맵을 준비
    - `이미지 전체 크기`, `이미지 절반 크기`, `이미지 1/3 크기`, `이미지 1/6 크기`
    - 출력 : 4096 x 60 x 60
3. Decoder 모듈(업샘플링 모듈)
    - Pyramid Pooling 모듈의 출력을 21 x 60 x 60(클래스 수 x 높이 x 폭)텐서로 변환
    - 21 x 60 x 60 (클래스 수 x 높이 x 폭)으로 변환된 텐서를 원 이미지 크기에 맞도록 변환(21 x 475 x 475)
    - 출력 : 21 x 475 x 475
    - 추론 시 Decoder 모듈의 출력으로 출력에 대한 최대 확률의 물체 클래스를 찾아 각 픽셀의 라벨을 결정.
4. AuxLoss (보조 loss)
    - 손실함수 게산을 보조 
    - Feature 모듈로 중간 정보를 빼내 입력 데이터로 사용
    - Decoder 모듈처럼 각 픽셀에 대응하는 물체 라벨 추정 클래스 분류 실행
    - 입력 : 1024 x 60 x 60, 출력 : 21 x 475 x 475
    - AuxLoss 모듈은 Feature 층의 중간까지 결과로 시맨틱 분할을 실시하여 분류 정확도는 떨어진다. 하지만 오차 역전파법 수행 시 Feature 층 중간까지의 네트워크 파라미터 학습을 보조하는 역할을 한다. 
    - 학습시에는 AuxLoss 모듈을 사용하지만 추론 시에는 AuxLoss 모듈의 출력은 사용하지 않고 Decoder 모듈의 출력만으로 semantic segmantation 을 실행한다.


## PSPNet 클래스
- PSPNet 클래스의 메서드는 forward 뿐.
- 순서대로 각 모듈의 서브 네트워크를 실행함. 
- 단 AuxLoss 모듈을 Feature 모듈의 네 번째 서브 네트워크 feature_dilated_res_1의 뒤에 넣어 output_aux변수로 출력을 작성
- forward 메서드 끝에 메인 output과 output_aux를 반환


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class PSPNet(nn.Module):
    def __init__(self, n_classes):
        super(PSPNet, self).__init__()

        # 파라미터 설정
        block_config = [3, 4, 6, 3]  # resnet50
        img_size = 475
        img_size_8 = 60  # ima/size의 1/8로 설정

        # 네 개의 모듈을 구성하는 서브 네트워크 준비
        self.feature_conv = FeatureMap_convolution()
        self.feature_res_1 = ResidualBlockPSP(
            n_blocks=block_config[0], in_channels=128, mid_channels=64, out_channels=256, stride=1, dilation=1)
        self.feature_res_2 = ResidualBlockPSP(
            n_blocks=block_config[1], in_channels=256, mid_channels=128, out_channels=512, stride=2, dilation=1)
        self.feature_dilated_res_1 = ResidualBlockPSP(
            n_blocks=block_config[2], in_channels=512, mid_channels=256, out_channels=1024, stride=1, dilation=2)
        self.feature_dilated_res_2 = ResidualBlockPSP(
            n_blocks=block_config[3], in_channels=1024, mid_channels=512, out_channels=2048, stride=1, dilation=4)

        self.pyramid_pooling = PyramidPooling(in_channels=2048, pool_sizes=[
            6, 3, 2, 1], height=img_size_8, width=img_size_8)

        self.decode_feature = DecodePSPFeature(
            height=img_size, width=img_size, n_classes=n_classes)

        self.aux = AuxiliaryPSPlayers(
            in_channels=1024, height=img_size, width=img_size, n_classes=n_classes)

    def forward(self, x):
        x = self.feature_conv(x)
        x = self.feature_res_1(x)
        x = self.feature_res_2(x)
        x = self.feature_dilated_res_1(x)

        output_aux = self.aux(x)  # Feature 모듈의 중간을 Aux 모듈로

        x = self.feature_dilated_res_2(x)

        x = self.pyramid_pooling(x)
        output = self.decode_feature(x)

        return (output, output_aux)

## Feature 모듈 설명 및 구현
- Feature 모듈은 다섯 개의 서브 네트워크인 `FeatureMap_convolution`, `두 개의 ResidualBlockPSP`, `두 개의 dilated(확장판) ResidualBlockPSP`로 구성됨
- 네 번째 서브 네트워크인 dilated 판 ResidualBlcokPSP의 출력 텐서 1024 x60 x 60(ch x 높이 x 폭)이 AuxLoss 모듈로 출력되는 점에 주의 
- AuxLoss모듈에서는 이 출력 텐서로 픽셀별 클래스를 분류하고 그 손실 값을 Feature 모듈의 전반부 네 개의 서브 네트워크를 학습하는데 사용.

###  서브 네트워크 FeatureMap_convolution
- 입력 : 3 x 475 x 475, 출력 : 128 x 119 x 119
- `convolution, batch noramalization, Relu를 한 층으로하는 conv2dBatchNormRlue가 세 층`, `max polling 한 층` 총 4층으로 이뤄져있음

In [ ]:
class conv2DBatchNormRelu(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, dilation, bias):
        super(conv2DBatchNormRelu, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels,
                              kernel_size, stride, padding, dilation, bias=bias)
        self.batchnorm = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        # inplace를 설정으로 입력을 저장하지 않고 출력을 게산하여 메모리 절약

    def forward(self, x):
        x = self.conv(x)
        x = self.batchnorm(x)
        outputs = self.relu(x)

        return outputs

In [ ]:
class FeatureMap_convolution(nn.Module):
    def __init__(self):
        '''構成するネットワークを用意'''
        super(FeatureMap_convolution, self).__init__()

        # 합성곱 층 1
        in_channels, out_channels, kernel_size, stride, padding, dilation, bias = 3, 64, 3, 2, 1, 1, False
        self.cbnr_1 = conv2DBatchNormRelu(
            in_channels, out_channels, kernel_size, stride, padding, dilation, bias)

        # 합성곱 층 2
        in_channels, out_channels, kernel_size, stride, padding, dilation, bias = 64, 64, 3, 1, 1, 1, False
        self.cbnr_2 = conv2DBatchNormRelu(
            in_channels, out_channels, kernel_size, stride, padding, dilation, bias)

        # 합성곱 층 3
        in_channels, out_channels, kernel_size, stride, padding, dilation, bias = 64, 128, 3, 1, 1, 1, False
        self.cbnr_3 = conv2DBatchNormRelu(
            in_channels, out_channels, kernel_size, stride, padding, dilation, bias)

        # 합성곱 층 4
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    def forward(self, x):
        x = self.cbnr_1(x)
        x = self.cbnr_2(x)
        x = self.cbnr_3(x)
        outputs = self.maxpool(x)
        return outputs

### ResidualBlockPSP
- bottleNeckPSP과 연속된 bottleNeckIdentifyPSP 로 이뤄져 있다.

In [ ]:
class ResidualBlockPSP(nn.Sequential):
    def __init__(self, n_blocks, in_channels, mid_channels, out_channels, stride, dilation):
        super(ResidualBlockPSP, self).__init__()

        # bottleNeckPSP 준비
        self.add_module(
            "block1",
            bottleNeckPSP(in_channels, mid_channels,
                          out_channels, stride, dilation)
        )

        # bottleNeckIdentifyPS 반복 준비
        for i in range(n_blocks - 1):
            self.add_module(
                "block" + str(i+2),
                bottleNeckIdentifyPSP(
                    out_channels, mid_channels, stride, dilation)
            )

In [ ]:
class conv2DBatchNorm(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, dilation, bias):
        super(conv2DBatchNorm, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels,
                              kernel_size, stride, padding, dilation, bias=bias)
        self.batchnorm = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = self.conv(x)
        outputs = self.batchnorm(x)

        return outputs

In [ ]:
class bottleNeckPSP(nn.Module):
    def __init__(self, in_channels, mid_channels, out_channels, stride, dilation):
        super(bottleNeckPSP, self).__init__()

        self.cbr_1 = conv2DBatchNormRelu(
            in_channels, mid_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)
        self.cbr_2 = conv2DBatchNormRelu(
            mid_channels, mid_channels, kernel_size=3, stride=stride, padding=dilation, dilation=dilation, bias=False)
        self.cb_3 = conv2DBatchNorm(
            mid_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)

        # 스킵 결합
        self.cb_residual = conv2DBatchNorm(
            in_channels, out_channels, kernel_size=1, stride=stride, padding=0, dilation=1, bias=False)

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        conv = self.cb_3(self.cbr_2(self.cbr_1(x)))
        residual = self.cb_residual(x)
        return self.relu(conv + residual)

In [ ]:
class bottleNeckIdentifyPSP(nn.Module):
    def __init__(self, in_channels, mid_channels, stride, dilation):
        super(bottleNeckIdentifyPSP, self).__init__()

        self.cbr_1 = conv2DBatchNormRelu(
            in_channels, mid_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)
        self.cbr_2 = conv2DBatchNormRelu(
            mid_channels, mid_channels, kernel_size=3, stride=1, padding=dilation, dilation=dilation, bias=False)
        self.cb_3 = conv2DBatchNorm(
            mid_channels, in_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        conv = self.cb_3(self.cbr_2(self.cbr_1(x)))
        residual = x
        return self.relu(conv + residual)

## Pyramid Pooling 모둘 설명 및 구현
- 입력 : 2048 x 60 x 60, 출력 : 4096 x 60 x 60
- 입력을 이용하여 총 5개의 특칭맵을 추출
- 5개 중 네 개는 출력이 각각 6, 3 ,2, 1인 Adptive Average Pooling 층으로 보내짐
- Average Pooling 층을 통과한 텐서는 `conv2DBatchNormRelu` 클래스를 지나 Upsample층에 도달
- upsample 층에서 모든 특징량의 크기를 입력 크기와 같은 60 x 60 크기로 확대
- 네 개의 분기는 채널 수가 각각 512개이고 512 x 4 + 2048 = 4096이 되어 `Pyramid Pooling`모듈의 출력 텐서 크기는 4096 x 90 x 90이다.
- 위 출력 텐서를 Decoder로 보냄.

In [ ]:
class PyramidPooling(nn.Module):
    def __init__(self, in_channels, pool_sizes, height, width):
        super(PyramidPooling, self).__init__()

        # forward에서 사용하는 이미지 크기
        self.height = height
        self.width = width

        # 각 합성곱 층의 출력 채널 수
        out_channels = int(in_channels / len(pool_sizes))

        # 각 합성곱 층 작성
        # 다음은 for 무능로 구현하는 것이 좋지만 이해를 돕기 위해 하드코딩
        # pool_sizes: [6, 3, 2, 1]
        self.avpool_1 = nn.AdaptiveAvgPool2d(output_size=pool_sizes[0])
        self.cbr_1 = conv2DBatchNormRelu(
            in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)

        self.avpool_2 = nn.AdaptiveAvgPool2d(output_size=pool_sizes[1])
        self.cbr_2 = conv2DBatchNormRelu(
            in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)

        self.avpool_3 = nn.AdaptiveAvgPool2d(output_size=pool_sizes[2])
        self.cbr_3 = conv2DBatchNormRelu(
            in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)

        self.avpool_4 = nn.AdaptiveAvgPool2d(output_size=pool_sizes[3])
        self.cbr_4 = conv2DBatchNormRelu(
            in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)

    def forward(self, x):

        out1 = self.cbr_1(self.avpool_1(x))
        out1 = F.interpolate(out1, size=(
            self.height, self.width), mode="bilinear", align_corners=True)

        out2 = self.cbr_2(self.avpool_2(x))
        out2 = F.interpolate(out2, size=(
            self.height, self.width), mode="bilinear", align_corners=True)

        out3 = self.cbr_3(self.avpool_3(x))
        out3 = F.interpolate(out3, size=(
            self.height, self.width), mode="bilinear", align_corners=True)

        out4 = self.cbr_4(self.avpool_4(x))
        out4 = F.interpolate(out4, size=(
            self.height, self.width), mode="bilinear", align_corners=True)

        # 최종 결합시킹 dim=1로 채널 수 차원에서 결합
        output = torch.cat([x, out1, out2, out3, out4], dim=1)


### Decoder, AuxLoss 모듈 설명 및 구현
- Pyramid Pooling 또는 Feature 모듈에서 출력된 텐서 정보를 Decode(해석)함
- 텐서 정보를 읽은 후 픽셀별로 물체 라벨을 클래스 분류로 추정하고 마지막 이미지 크기를 원래 475 x475로 업샘플링 한다. 
- Decoder과 AuxLoss 모듈은 모두 동일한 네트워크 구성이다.
- `conv2DBatchNormRelu`클래스를 통과한 후 dropout 층을 지나 합성곱층을 통과 하여 텐서 크기가 21 x 60 x 60이 된다. 
- 마지막으로 UpSample 층을 지나 PSPNet 입력 화상의 크기였던 475 사이즈로 확대한다. 
- Upsample은 F.interpolate 연산으로 구현

In [ ]:
class DecodePSPFeature(nn.Module):
    def __init__(self, height, width, n_classes):
        super(DecodePSPFeature, self).__init__()

        # forward에 사용하는 이미지 크기
        self.height = height
        self.width = width

        self.cbr = conv2DBatchNormRelu(
            in_channels=4096, out_channels=512, kernel_size=3, stride=1, padding=1, dilation=1, bias=False)
        self.dropout = nn.Dropout2d(p=0.1)
        self.classification = nn.Conv2d(
            in_channels=512, out_channels=n_classes, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        x = self.cbr(x)
        x = self.dropout(x)
        x = self.classification(x)
        output = F.interpolate(
            x, size=(self.height, self.width), mode="bilinear", align_corners=True)

        return output

In [ ]:
class AuxiliaryPSPlayers(nn.Module):
    def __init__(self, in_channels, height, width, n_classes):
        super(AuxiliaryPSPlayers, self).__init__()

        # forward에 사용하는 이미지 크기
        self.height = height
        self.width = width

        self.cbr = conv2DBatchNormRelu(
            in_channels=in_channels, out_channels=256, kernel_size=3, stride=1, padding=1, dilation=1, bias=False)
        self.dropout = nn.Dropout2d(p=0.1)
        self.classification = nn.Conv2d(
            in_channels=256, out_channels=n_classes, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        x = self.cbr(x)
        x = self.dropout(x)
        x = self.classification(x)
        output = F.interpolate(
            x, size=(self.height, self.width), mode="bilinear", align_corners=True)

        return output

In [ ]:
# 모델 정의
net = PSPNet(n_classes=21)
net

PSPNet(
  (feature_conv): FeatureMap_convolution(
    (cbnr_1): conv2DBatchNormRelu(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (cbnr_2): conv2DBatchNormRelu(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (cbnr_3): conv2DBatchNormRelu(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (feature_res_1): ResidualBlockPSP(
    (block1): bottleNec

# fine 튜닝 학습 및 점증

In [ ]:
import random
import math
import time
import pandas as pd
import numpy as np

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# Setup seeds
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

## 데이터로더 생성

In [ ]:
from utils.dataloader import make_datapath_list, DataTransform, VOCDataset

# 파일 경로 릿트 생성
rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath=rootpath)

# 데이터셋 작성
# (RGB)색의 평균값과 표준편차
color_mean = (0.485, 0.456, 0.406)
color_std = (0.229, 0.224, 0.225)

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(
    input_size=475, color_mean=color_mean, color_std=color_std))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size=475, color_mean=color_mean, color_std=color_std))

# DataLoader작성
batch_size = 8

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# dict로 저장
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

In [ ]:
from utils.pspnet import PSPNet

# 파인튜닝으로 PSPNet 생성
# ADE20k 데이터셋의 학습된 모델을 사용하며 ADE20k 클래스 수는 150
net = PSPNet(n_classes=150)

# ADE20K학습된 파라미터 읽기
state_dict = torch.load("./pspnet50_ADE20K.pth")
net.load_state_dict(state_dict)

# 분류용 합성곱 층을 출력 수 21로 바꾼다.
n_classes = 21
net.decode_feature.classification = nn.Conv2d(
    in_channels=512, out_channels=n_classes, kernel_size=1, stride=1, padding=0)

net.aux.classification = nn.Conv2d(
    in_channels=256, out_channels=n_classes, kernel_size=1, stride=1, padding=0)

# 교체한 합성곱 층 초기화. 활성화 함수는 시그모이드 함수이므로 Xavier 사용


def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_normal_(m.weight.data)
        if m.bias is not None:  # 바이어스가 있는 경우
            nn.init.constant_(m.bias, 0.0)

# https://drive.google.com/open?id=12eN6SpnawYuQmD1k9VgVW3QSgPR6hICc (weight 다운)
net.decode_feature.classification.apply(weights_init)
net.aux.classification.apply(weights_init)


print('네트워크 설정 완료 : pre-trained 가중치 load')

## 손실함수 정의
- cross entropy loss  와 auxloss 손실 합을 총 손실로함
- AuxLOss는 계수 0.4를 곱하여 가중치를 메인 손실보다 작게한다.

In [ ]:
# 손실함수 정의
class PSPLoss(nn.Module):
    """PSPNet 손실함수 클래스 """

    def __init__(self, aux_weight=0.4):
        super(PSPLoss, self).__init__()
        self.aux_weight = aux_weight  # aux_loss가중치

    def forward(self, outputs, targets):
        """
        손실함수 계산

        Parameters
        ----------
        outputs : PSPNet 출력(tuple)
            (output=torch.Size([num_batch, 21, 475, 475]), output_aux=torch.Size([num_batch, 21, 475, 475]))。

        targets : [num_batch, 475, 475]
            정답 어노테이션 정보

        Returns
        -------
        loss : 텐서
            손실 값
        """

        loss = F.cross_entropy(outputs[0], targets, reduction='mean')
        loss_aux = F.cross_entropy(outputs[1], targets, reduction='mean')

        return loss+self.aux_weight*loss_aux


criterion = PSPLoss(aux_weight=0.4)

## 스케줄러로 에폭별 학습 비율 변경
---
- 입력에 가까운 모듈의 학습률은 작게, 교체한 합성곱 층을 가진 Deocder와 AuxLoss 모듈은 크게 설정
- 에폭 별 학습률을 변화시키는 스케쥴러 활용 
- lambda_epoch함수는 최대 에폭 수를 30으로하고 에폭을 거칠 때마다 학습률이 서서히 작아짐.
- return 하는 값을 옵티마이저 학습률에 곱한다. 

In [ ]:
# 파인튜닝이므로 학습률 작게
optimizer = optim.SGD([
    {'params': net.feature_conv.parameters(), 'lr': 1e-3},
    {'params': net.feature_res_1.parameters(), 'lr': 1e-3},
    {'params': net.feature_res_2.parameters(), 'lr': 1e-3},
    {'params': net.feature_dilated_res_1.parameters(), 'lr': 1e-3},
    {'params': net.feature_dilated_res_2.parameters(), 'lr': 1e-3},
    {'params': net.pyramid_pooling.parameters(), 'lr': 1e-3},
    {'params': net.decode_feature.parameters(), 'lr': 1e-2},
    {'params': net.aux.parameters(), 'lr': 1e-2},
], momentum=0.9, weight_decay=0.0001)


# 스케쥴러 설정
def lambda_epoch(epoch):
    max_epoch = 30
    return math.pow((1-epoch/max_epoch), 0.9)


scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_epoch)

## train 정의
1. 스케쥴러 적용
2. 멀티플 미니 배치 적용
    - 손실 loss의 계산과 경사를 구하는 backward를 여러 번 실행 .
    - 각 파라미터에 복수의 미니 배치로 계산된 경사의 합계를 구하여 optimizer.step()를 실행한 후 파라미터를 갱신하는 기술

In [ ]:
# 모델을 학습시키는 함수


def train_model(net, dataloaders_dict, criterion, scheduler, optimizer, num_epochs):

    # Gpu를 사용할 수 있는지 확인
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("사용 장치：", device)

    # 네트워크를 gpu로
    net.to(device)

    # 네트워크가 어느 정도 고정되면 고속화
    torch.backends.cudnn.benchmark = True

    # 이미지 갯수
    num_train_imgs = len(dataloaders_dict["train"].dataset)
    num_val_imgs = len(dataloaders_dict["val"].dataset)
    batch_size = dataloaders_dict["train"].batch_size

    # iteratior의 카운터 설정
    iteration = 1
    logs = []

    # multiple minibatch
    batch_multiplier = 3

    
    for epoch in range(num_epochs):

        # 시작 시간 저장
        t_epoch_start = time.time()
        t_iter_start = time.time()
        epoch_train_loss = 0.0  # epoch손실합
        epoch_val_loss = 0.0  # epoch손실합

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epoch 학습 및 검증
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # 모델을 학습 모드로
                scheduler.step()  # optimizer  스케쥴러 갱신
                optimizer.zero_grad()
                print('（train）')

            else:
                if((epoch+1) % 5 == 0):
                    net.eval()   # 모델을 검증 모드로
                    print('-------------')
                    print('（val）')
                else:
                    # 검증은 다섯 번 중 한번만 수행
                    continue

            # 데이터 로더에서 미니 배치를 꺼내 수행
            count = 0  # multiple minibatch
            for imges, anno_class_imges in dataloaders_dict[phase]:
                # 미니 배치 크기가 1이면 배치 정규화에서 오류가 발생하여 피한다.
                
                if imges.size()[0] == 1:
                    continue

                # gpu를 사용할 수 있으면 gpu에 데이터를 보낸다
                imges = imges.to(device)
                anno_class_imges = anno_class_imges.to(device)

                
                # multiple minibatch로 파라미터 갱신
                if (phase == 'train') and (count == 0):
                    optimizer.step()
                    optimizer.zero_grad()
                    count = batch_multiplier

                # 순전파 계산
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(imges)
                    loss = criterion(
                        outputs, anno_class_imges.long()) / batch_multiplier

                    # 훈련시 역전파
                    if phase == 'train':
                        loss.backward()  # 경사 계산
                        count -= 1  # multiple minibatch

                        if (iteration % 10 == 0):  # 10iter에 한 번 loss 표시
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('반복 {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                iteration, loss.item()/batch_size*batch_multiplier, duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item() * batch_multiplier
                        iteration += 1

                    # 검증
                    else:
                        epoch_val_loss += loss.item() * batch_multiplier

        # epoch의phase별 손실과 정답률
        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss/num_train_imgs, epoch_val_loss/num_val_imgs))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

        # 로그 저장
        log_epoch = {'epoch': epoch+1, 'train_loss': epoch_train_loss /
                     num_train_imgs, 'val_loss': epoch_val_loss/num_val_imgs}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("log_output.csv")

    # 마지막 모델 저장
    torch.save(net.state_dict(), 'weights/pspnet50_' +
               str(epoch+1) + '.pth')

In [ ]:
# 학습 및 검증 실행
num_epochs = 30
train_model(net, dataloaders_dict, criterion, scheduler, optimizer, num_epochs=num_epochs)

## inference 
- 파일 경로 리스트를 만듦
- VOC 데이터셋 이미지가 아니라 승마 이미지에 대해 추론하며 어노테이션 이미지를 한장 사용
- 어노테이션 이미지를 사용하는 이유는 2가지가 있다.
    1. 어노에티셔 이미지가 없으면 전처리 클래스의 함수가 제대로 작동하지 않는다. 실제로 추론에 사용하지 않지만 더미 데이터로서 함수에 전달 된다.
    2. 어노테이션 이미지에서 색상 팔레트 정보를 추출하지 않으면 물체 라벨에 해당하는 생상 정보가 존재하지 않게 된다. 

In [ ]:
cd pytorch_advanced/2_objectdetection/

/content/pytorch_advanced/2_objectdetection


In [ ]:
from utils.dataloader import make_datapath_list, DataTransform


# 파일 경로 리스트 작성|
rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath=rootpath)

In [ ]:
#psp net 준비
from utils.pspnet import PSPNet

net = PSPNet(n_classes=21)

# 학습된 파라미터 읽기
state_dict = torch.load("./weights/pspnet50_30.pth",
                        map_location={'cuda:0': 'cpu'})
net.load_state_dict(state_dict)

print('네트워크 설정 및 가중치 로딩 완료.')

In [ ]:
 # 1. 원본 이미지 표시
image_file_path = "./data/cowboy-757575_640.jpg"
img = Image.open(image_file_path)   # [높이][폭][색RGB]
img_width, img_height = img.size
plt.imshow(img)
plt.show()

# 2. 전처리 클래스 작성
color_mean = (0.485, 0.456, 0.406)
color_std = (0.229, 0.224, 0.225)
transform = DataTransform(
    input_size=475, color_mean=color_mean, color_std=color_std)

# 3. 전처리
# 적당한 어노테이션 이미지를 준비하여 생상 팔레트 정보 추출
anno_file_path = val_anno_list[0]
anno_class_img = Image.open(anno_file_path)   # [높이][폭]
p_palette = anno_class_img.getpalette()
phase = "val"
img, anno_class_img = transform(phase, img, anno_class_img)


# 4. PSPNet로 추론
net.eval()
x = img.unsqueeze(0)  # 미니 배치화：torch.Size([1, 3, 475, 475])
outputs = net(x)
y = outputs[0]  # AuxLoss 는 무시  y의 크기 : torch.Size([1, 21, 475, 475])


# 5. PSPNet 출력으로 최대 클래스를 구하여 생상 팔레트 형식으로 이미지 크기를 원래대로 되돌림.
y = y[0].detach().numpy()  # y：torch.Size([1, 21, 475, 475])
y = np.argmax(y, axis=0)
anno_class_img = Image.fromarray(np.uint8(y), mode="P")
anno_class_img = anno_class_img.resize((img_width, img_height), Image.NEAREST)
anno_class_img.putpalette(p_palette)
plt.imshow(anno_class_img)
plt.show()


# 6. 이미지를 투과시켜 겹친다.
trans_img = Image.new('RGBA', anno_class_img.size, (0, 0, 0, 0))
anno_class_img = anno_class_img.convert('RGBA')  # 색상 팔레트 형식을 RGBA로 변경

for x in range(img_width):
    for y in range(img_height):
        # 추론 결과 화상의 픽셀 데이터 취득
        pixel = anno_class_img.getpixel((x, y))
        r, g, b, a = pixel

        # (0, 0, 0)의 배경이라면 그대로 투과
        if pixel[0] == 0 and pixel[1] == 0 and pixel[2] == 0:
            continue
        else:
            # 그 외 색상은 준비된 화상에 픽셀 기록
            trans_img.putpixel((x, y), (r, g, b, 150))
            # 투과율을 150으로 지정

img = Image.open(image_file_path)   # [높이][폭][색RGB]
result = Image.alpha_composite(img.convert('RGBA'), trans_img)
plt.imshow(result)
plt.show()

시맨틱 분할의 정확성을 높이려면 학습 에폭 수를 늘려야함 
